In [ ]:
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from PIL import Image
import polars as pl
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import numpy as np

# ============================================================
# Download do dataset OASIS via kagglehub
# ============================================================
path = kagglehub.dataset_download("ninadaithal/imagesoasis")

# Converter dataset de imagens em DataFrame Polars para EDA
data_path = Path(path) / "Data"

# Coletar paths das imagens usando Polars
image_paths = []
for p in data_path.rglob('*.*'):
    if p.is_file():
        parts = p.parts
        if len(parts) >= 2:
            class_name = parts[-2]  # pasta = classe
            file_name = parts[-1]   # nome do arquivo
            image_paths.append((class_name, file_name, str(p)))

print(f"Encontradas {len(image_paths)} imagens")

# Criar DataFrame Polars
df = pl.DataFrame({
    'class': [x[0] for x in image_paths],
    'image': [x[1] for x in image_paths],
    'path': [x[2] for x in image_paths]
})

# Salvar metadados em arquivo Parquet
output_path = "oasis_dataset_metadata.parquet"
df.write_parquet(output_path)


# ============================================================
# Configurações básicas
# ============================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo: {device}")

# Transformações de pré-processamento
transform = transforms.Compose([
    transforms.Resize((248, 496)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset personalizado
class OASISDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.class_to_idx = {
            'Non Demented': 0,
            'Very mild Dementia': 1,
            'Mild Dementia': 2,
            'Moderate Dementia': 3
        }

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.row(idx)
        img_path = row[2]
        class_name = row[0]

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.class_to_idx[class_name]
        return image, label

# Carregar metadados
df = pl.read_parquet("oasis_dataset_metadata.parquet")
dataset = OASISDataset(df, transform=transform)

# Split treino/validação
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


# ============================================================
# Modelo AlexNet (torchvision)
# ============================================================
model = models.alexnet(weights=None)
# Ajustar a última camada para 4 classes
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 4)
model = model.to(device)

# Loss function com pesos (para classes desbalanceadas)
class_weights = torch.tensor([0.3215, 1.5744, 4.3201, 44.2812], dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


# ============================================================
# Treinamento
# ============================================================
num_epochs = 5
metrics_history = []

start_time = time.time()
for epoch in range(num_epochs):
    epoch_start = time.time()

    # --- Training ---
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Estatísticas
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    epoch_time = time.time() - epoch_start

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')
    print(f'Acc: {acc:.4f} | Prec: {prec:.4f} | Recall: {rec:.4f} | F1-macro: {f1:.4f}')
    print('-' * 60)

    # Guardar métricas
    metrics_history.append({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "accuracy": acc,
        "precision_macro": prec,
        "recall_macro": rec,
        "f1_macro": f1,
        "epoch_time_sec": epoch_time
    })

    scheduler.step()

print("Treinamento concluído!")

# ============================================================
# Salvar métricas em CSV
# ============================================================
metrics_df = pd.DataFrame(metrics_history)
metrics_df.to_csv("alexnet_metrics.csv", index=False)
print("Métricas salvas em: alexnet_metrics.csv")

# --- Após o treino ou validação final ---
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calcular a matriz de confusão
cm = confusion_matrix(all_labels, all_preds)
print("Matriz de Confusão:\n", cm)

# Salvar com numpy
np.save("alexnet_confusion_matrix.npy", cm)
print("Matriz de confusão salva em alexnet_confusion_matrix.npy")

Using Colab cache for faster access to the 'imagesoasis' dataset.
Encontradas 86437 imagens
Dispositivo: cuda
Epoch 1/5
Train Loss: 1.2248 | Val Loss: 0.9394
Acc: 0.5158 | Prec: 0.3703 | Recall: 0.6444 | F1-macro: 0.3587
------------------------------------------------------------
Epoch 2/5
Train Loss: 0.5861 | Val Loss: 0.3063
Acc: 0.8567 | Prec: 0.7496 | Recall: 0.9083 | F1-macro: 0.8119
------------------------------------------------------------
Epoch 3/5
Train Loss: 0.2318 | Val Loss: 0.1191
Acc: 0.9452 | Prec: 0.9250 | Recall: 0.9658 | F1-macro: 0.9436
------------------------------------------------------------
Epoch 4/5
Train Loss: 0.1171 | Val Loss: 0.0717
Acc: 0.9555 | Prec: 0.9360 | Recall: 0.9781 | F1-macro: 0.9550
------------------------------------------------------------
Epoch 5/5
Train Loss: 0.0608 | Val Loss: 0.0545
Acc: 0.9637 | Prec: 0.9134 | Recall: 0.9847 | F1-macro: 0.9437
------------------------------------------------------------
Treinamento concluído!
Métrica